In [1]:
import os, sys
import argparse
import time

import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

if sys.platform == "win32":
    # windows, win32
    from sumolib import checkBinary
else:
    # mac, darwin
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        sys.exit("please declare environment variable 'SUMO_HOME'")
    
import traci

In [2]:
import curbside
import utils

In [3]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [7]:
def simulate(seconds = 10, gui = False):
    """
    Main function that controls the simulation
    """

    road_network = utils.create_graph('seattle.net.xml')
    
    # create agent curbs
    curb_ids = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curb_ids.append(kid.get('id'))
    
    curbs = {}
    for curb_id in curb_ids:
        curbs[curb_id] = curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'delivery'], road_network)
        
    for curb in curbs.values():
        curb.find_neighborhood(road_network, curbs)
        curb.dlv_cap = curb.tot_cap
    
    reroute_cost = {}
    for curb_id in curb_ids:
        reroute_cost[curb_id] = 0
    
    if sys.platform == "win32":
        if gui:
            sumoBinary = checkBinary('sumo-gui')
        else:
            sumoBinary = checkBinary('sumo')
    
    else:
        # mac, darwin
        if gui:
            sumoBinary = "sumo-gui"
        else:
            sumoBinary = "sumo"
    
    traci.start([sumoBinary, "-c", "seattle.sumocfg", "--time-to-teleport", "-1"])
    
    start = time.time()
    # simulation
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        if traci.simulation.getTime() == seconds:
            run_time = time.time() - start
            break
        
        for curb_id, curb in curbs.items():
            
            # colelct neighbor info from last step
            curb._update_neigh_occupy(curbs)
            
            # check if there is any vehicle enters
            v_enter = set(traci.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
            
            # no need to subtract v_leave, because v_leave ideally should be maintained per curb, but now inner loop is on curbs
            # v_enter -= v_leave
            
            for veh in v_enter:
                # new vehicle is for sure not confirmed, no need to check confirmed=True
                # if vehicle has next stops and currect edge is one of them - this vehicle should be considered for current edge
                # also check vehicle is not stopped - during parking right before departure getNextStops() will return original travel plan, which shouldn't happen
                
                if traci.vehicle.getNextStops(veh) and curb.id in [item[2] for item in traci.vehicle.getNextStops(veh)] and not traci.vehicle.isStopped(veh):
                    # if the trips is not ending and the current edge is the parking stop
                    # v_leave should be excluded
                    
                    # vclass: passenger or delivery
                    vclass = traci.vehicle.getVehicleClass(veh)
                    if vclass == 'delivery':
                        occ = curb.dlv_occ
                        cap = curb.dlv_cap
                    else:
                        occ = curb.psg_occ
                        cap = curb.psg_cap
                        
                    if occ < cap:
                        # if can park, add to occupied set : planned + parked
                        curb.occupied_vehicle.add(veh)
                        
                    else:
                        # cannot park at this edge, reroute
                        # item._reroute_choice() returns (curb_id, distance) tuple
                        reroute_msg = curb._reroute_choice(veh, curbs)
                        traci.vehicle.rerouteParkingArea(veh, reroute_msg[0])
                        reroute_cost[curb_id] += reroute_msg[1]
                        
            # v_leave : parked vehicle of last time step - parked vehicle at this time step
            v_leave = curb.parked_vehicle - set(traci.parkingarea.getVehicleIDs(curb.id))
            
            # remove v_leave from occupied set : planned + parked
            curb.occupied_vehicle -= v_leave
            
            # update parked vehicle set
            curb.parked_vehicle = set(traci.parkingarea.getVehicleIDs(curb.id))
            
            # update moving vehicles on the hosting edge
            curb.moving_vehicle = set(traci.edge.getLastStepVehicleIDs(curb.edge))
            
            # update two-type occupancy for next time step
            curb._occupy_cnt()

    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    
    return reroute_cost, run_time

In [14]:
utils.generate_route(psg_park_dm_x=5, dlv_park_dm_x=2, flow_num=200, simulate_hour=12, seed=2)
cost, run_time = simulate(seconds = 3600, gui = False)

seattle.trips.xml re-generated. (demand factor = 5 )
 Retrying in 1 seconds


In [15]:
run_time

126.10756421089172

In [16]:
cost

{'01-24-NE': 153.66,
 '01-24-SW': 384.15,
 '01-25-NE': 0,
 '01-25-SW': 81.69,
 '01-26-SW': 157.48,
 '02-24-NE': 232.17000000000002,
 '02-24-SW': 232.17000000000002,
 '02-25-NE': 0,
 '02-25-SW': 82.95,
 '02-26-NE': 313.92,
 '02-26-SW': 0,
 '03-24-NE': 153.54,
 '03-24-SW': 0,
 '03-25-NE': 0,
 '03-25-SW': 0,
 '03-26-SW': 321.72,
 '04-24-NE': 0,
 '04-24-SW': 0,
 '04-25-NE': 0,
 '04-25-SW': 0,
 '04-26-NE': 0,
 '04-26-SW': 0,
 '24-01-NW': 0,
 '24-01-SE': 0,
 '24-02-NW': 156.59,
 '24-02-SE': 0,
 '24-03-NW': 0,
 '25-01-NW': 0,
 '25-01-SE': 0,
 '25-02-NW': 0,
 '25-02-SE': 79.49,
 '25-03-NW': 0,
 '26-01-NW': 238.62,
 '26-01-SE': 0,
 '26-02-NW': 0,
 '26-02-SE': 0,
 '26-03-NW': 0,
 '26-03-SE': 0,
 '27-01-NW': 236.04000000000002,
 '27-01-SE': 0,
 '27-02-NW': 78.78,
 '27-02-SE': 78.78,
 '27-03-NW': 77.97,
 '27-03-SE': 311.88}